In [1]:
import numpy as np
import tensorflow as tf

In [3]:
a=np.random.randint(5,8,10)
print(a)
print(a[:-1])

[7 6 5 6 7 6 5 5 5 7]
[7 6 5 6 7 6 5 5 5]


In [55]:
def pad(seq,max_len):
    padding=np.full((len(seq),max_len),0,dtype=np.float32)
    for i,seq in enumerate(seq):
        padding[i,:len(seq)]=seq
    return padding
c=[[1,2,3],[4,5,6],[7,8,9]]
print(pad(c,5))

[[1. 2. 3. 0. 0.]
 [4. 5. 6. 0. 0.]
 [7. 8. 9. 0. 0.]]


In [3]:
a=np.arange(5)[:,None]
b=np.arange(5)[None,:]
print(a,b)

[[0]
 [1]
 [2]
 [3]
 [4]] [[0 1 2 3 4]]


In [4]:
a*b


array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12],
       [ 0,  4,  8, 12, 16]])

In [6]:
import math
a=math.sqrt(5.00000000000000000000)
print(a)

2.23606797749979


In [2]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import numpy as np
a=np.arange(5)
a=tf.constant([1,2,3],dtype=tf.float32)
print(a)

Tensor("Const:0", shape=(3,), dtype=float32)


In [5]:
import  sys
sys.path


['E:\\Desktop\\PythonProject\\NLP-Tutorials-master',
 'E:\\Anaconda3\\envs\\BERT\\python36.zip',
 'E:\\Anaconda3\\envs\\BERT\\DLLs',
 'E:\\Anaconda3\\envs\\BERT\\lib',
 'E:\\Anaconda3\\envs\\BERT',
 '',
 'E:\\Anaconda3\\envs\\BERT\\lib\\site-packages',
 'E:\\Anaconda3\\envs\\BERT\\lib\\site-packages\\win32',
 'E:\\Anaconda3\\envs\\BERT\\lib\\site-packages\\win32\\lib',
 'E:\\Anaconda3\\envs\\BERT\\lib\\site-packages\\Pythonwin',
 'E:\\Anaconda3\\envs\\BERT\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\China\\.ipython',
 'D:\\Program Files\\JetBrains\\PyCharm 2021.1.1\\plugins\\python\\helpers\\pydev',
 'D:\\Program Files\\JetBrains\\PyCharm 2021.1.1\\plugins\\python\\helpers-pro\\jupyter_debug']

In [32]:
import numpy as np
import datetime
PAD_ID = 0
class DateData:
    def __init__(self, n):
        #np.random.seed(1)
        self.date_cn = []
        self.date_en = []
        for timestamp in np.random.randint(143835585, 2043835585, n):
            date = datetime.datetime.fromtimestamp(timestamp)
            self.date_cn.append(date.strftime("%y-%m-%d"))
            self.date_en.append(date.strftime("%d/%b/%Y"))
        self.vocab = set(
            [str(i) for i in range(0, 10)] + ["-", "/", "<GO>", "<EOS>"] + [
                i.split("/")[1] for i in self.date_en])
        self.v2i = {v: i for i, v in enumerate(sorted(list(self.vocab)), start=1)}
        self.v2i["<PAD>"] = PAD_ID
        self.vocab.add("<PAD>")
        self.i2v = {i: v for v, i in self.v2i.items()}
        self.x, self.y = [], []
        for cn, en in zip(self.date_cn, self.date_en):
            self.x.append([self.v2i[v] for v in cn])
            self.y.append(
                [self.v2i["<GO>"], ] + [self.v2i[v] for v in en[:3]] + [
                    self.v2i[en[3:6]], ] + [self.v2i[v] for v in en[6:]] + [
                    self.v2i["<EOS>"], ])
        self.x, self.y = np.array(self.x), np.array(self.y)
        self.start_token = self.v2i["<GO>"]
        self.end_token = self.v2i["<EOS>"]

    def sample(self, n=64):
        bi = np.random.randint(0, len(self.x), size=n)
        bx, by = self.x[bi], self.y[bi]
        decoder_len = np.full((len(bx),), by.shape[1] - 1, dtype=np.int32)
        return bx, by, decoder_len

    def idx2str(self, idx):
        x = []
        for i in idx:
            x.append(self.i2v[i])
            if i == self.end_token:
                break
        return "".join(x)

    @property
    def num_word(self):
        return len(self.vocab)

In [82]:
d=DateData(4000)
print(d.num_word)
a,b,_=d.sample(5)
print(d.idx2str(a[1]),d.idx2str(b[1]))

27
09-06-02 <GO>02/Jun/2009<EOS>


In [49]:
import transformer
from tensorflow import keras
class PositionEmbedding(keras.layers.Layer):

    def __init__(self, max_len, model_dim, n_vocab):
        super(PositionEmbedding, self).__init__()
        pos = np.arange(max_len)[:, None]
        pe = pos / np.power(10000, 2. * np.arange(model_dim)[None, :] / model_dim)  # [max_len, dim]
        pe[:, 0::2] = np.sin(pe[:, 0::2])
        pe[:, 1::2] = np.cos(pe[:, 1::2])
        pe = pe[None, :, :]  # [1, max_len, model_dim]    for batch adding
        self.pe = tf.constant(pe, dtype=tf.float32)
        self.embeddings = keras.layers.Embedding(
            input_dim=n_vocab, output_dim=model_dim,  # [n_vocab, dim]
            embeddings_initializer=tf.initializers.RandomNormal(0,0.01)
        )

    def call(self, x):
        x_embed = self.embeddings(x) + self.pe  # [n, step, dim]
        return x_embed

def pad_zero(seqs, max_len):
    padded = np.full((len(seqs), max_len), fill_value=PAD_ID, dtype=np.int32)
    for i, seq in enumerate(seqs):
        padded[i, :len(seq)] = seq
    return padded

po=PositionEmbedding(12,32,d.num_word)
print(po.call(pad_zero(a,12)))

tf.Tensor(
[[[-0.00607561  1.0091705  -0.00723911 ...  0.9888509   0.00440611
    0.987182  ]
  [ 0.83539534  0.8551797   0.3037445  ...  0.9888509   0.00440614
    0.987182  ]
  [ 0.93363637  0.42383984  0.5825137  ...  1.015552    0.00427375
    1.0144999 ]
  ...
  [ 0.39555362  0.34382358  0.3033074  ...  0.9980838   0.01377204
    1.0090394 ]
  [-0.560586    0.7922952  -0.00863523 ...  0.9980838   0.01377208
    1.0090394 ]
  [-1.0165551   0.9974206  -0.31852672 ...  0.9980838   0.01377211
    1.0090394 ]]

 [[ 0.00876802  0.98913515 -0.00881294 ...  0.9962374   0.0160694
    1.023641  ]
  [ 0.83539534  0.8551797   0.3037445  ...  0.9888509   0.00440614
    0.987182  ]
  [ 0.93363637  0.42383984  0.5825137  ...  1.015552    0.00427375
    1.0144999 ]
  ...
  [ 0.39555362  0.34382358  0.3033074  ...  0.9980838   0.01377204
    1.0090394 ]
  [-0.560586    0.7922952  -0.00863523 ...  0.9980838   0.01377208
    1.0090394 ]
  [-1.0165551   0.9974206  -0.31852672 ...  0.9980838   0.01377

In [56]:
MODEL_DIM = 32
MAX_LEN = 12
N_LAYER = 3
N_HEAD = 4
DROP_RATE = 0.1
tr=transformer.Transformer(MODEL_DIM, MAX_LEN, N_LAYER, N_HEAD, d.num_word, DROP_RATE)
a,b=pad_zero(a,12),pad_zero(b,13)
logits=tr.call(a,b[:,:-1],training=True)#5,12,27
pad_mask = tf.math.not_equal(b[:, 1:], 0)
print(pad_mask)
print(logits)

tf.Tensor(
[[ True  True  True  True  True  True  True  True  True  True False False]
 [ True  True  True  True  True  True  True  True  True  True False False]
 [ True  True  True  True  True  True  True  True  True  True False False]
 [ True  True  True  True  True  True  True  True  True  True False False]
 [ True  True  True  True  True  True  True  True  True  True False False]], shape=(5, 12), dtype=bool)
tf.Tensor(
[[[ 7.01694965e-01  8.84610355e-01 -5.70255697e-01 ... -3.82010974e-02
   -7.29324758e-01 -5.36811590e-01]
  [ 1.31636190e+00  1.94019422e-01  4.47439253e-01 ...  5.28426826e-01
   -8.06298077e-01  4.79525954e-01]
  [ 9.16556001e-01  7.47860909e-01  3.49119127e-01 ...  1.90103188e-01
   -8.31119001e-01  1.25112101e-01]
  ...
  [ 1.05404913e+00  7.90725291e-01 -5.36957085e-02 ...  1.23864621e-01
   -2.95139700e-01  2.99802423e-01]
  [ 8.20983231e-01  8.46625745e-01  7.16648474e-02 ...  6.81980014e-01
   -3.23431402e-01  1.86402664e-01]
  [ 4.50143665e-01  4.14485097e-0

In [57]:
cross_entropy = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction="none")
loss=cross_entropy(b[:,1:],logits)
print(loss)

tf.Tensor(
[[4.2274036 5.303242  3.3451223 3.6147566 3.467957  4.294554  3.998591
  2.9467964 2.9518647 3.5703099 3.0589213 3.2146697]
 [3.2386942 3.5481462 2.8879046 4.1129684 2.7964873 3.5853252 4.010725
  2.7821813 1.859833  3.921908  3.1102905 3.0418892]
 [3.9510837 4.8487387 3.592229  3.5697064 4.275025  3.290856  4.272262
  3.9371936 4.312956  3.8514233 2.833174  2.4806342]
 [4.3538256 4.712538  3.7908401 4.3980675 3.1491795 3.0805075 4.092675
  4.5235863 4.5434456 2.7644482 3.1589518 3.3833327]
 [3.7048922 3.4166017 3.925446  3.7372298 3.8103368 3.9289277 4.356782
  3.9929988 3.3828325 3.8056679 2.3186758 3.1961021]], shape=(5, 12), dtype=float32)


In [74]:
a=pad_zero(a,15)
print(a)
def _pad_bool(seqs):
    return tf.math.equal(seqs, 0)

def _pad_mask(seqs):
    mask = tf.cast(_pad_bool(seqs), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]  # (n, 1, 1, step)
print(_pad_mask(a))

[[ 5  8  1  3 12  1  4 12  0  0  0  0  0  0  0]
 [10 10  1  3  4  1  4 12  0  0  0  0  0  0  0]
 [ 5  7  1  3  5  1  5  7  0  0  0  0  0  0  0]
 [ 6  4  1  3 10  1  5  6  0  0  0  0  0  0  0]
 [ 5  3  1  4  3  1  3  6  0  0  0  0  0  0  0]]
tf.Tensor(
[[[[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]]]


 [[[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]]]


 [[[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]]]


 [[[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]]]


 [[[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]]]], shape=(5, 1, 1, 15), dtype=float32)


In [78]:
import numpy as np
a=[[[0,0,1,1]]]
c=np.zeros((2,2,4))
print(c+a)

[[[0. 0. 1. 1.]
  [0. 0. 1. 1.]]

 [[0. 0. 1. 1.]
  [0. 0. 1. 1.]]]


In [81]:
def _look_ahead_mask( seqs):
    mask = 1 - tf.linalg.band_part(tf.ones((5, 5)), -1, 0)
    mask = tf.where(_pad_bool(seqs)[:, tf.newaxis, tf.newaxis, :], 1, mask[tf.newaxis, tf.newaxis, :, :])
    return mask  # (step, step)  n,1,step,step
c=[[1,2,3,0,0],[2,3,4,0,0]]
print(_look_ahead_mask(c))

tf.Tensor(
[[[[0. 1. 1. 1. 1.]
   [0. 0. 1. 1. 1.]
   [0. 0. 0. 1. 1.]
   [0. 0. 0. 1. 1.]
   [0. 0. 0. 1. 1.]]]


 [[[0. 1. 1. 1. 1.]
   [0. 0. 1. 1. 1.]
   [0. 0. 0. 1. 1.]
   [0. 0. 0. 1. 1.]
   [0. 0. 0. 1. 1.]]]], shape=(2, 1, 5, 5), dtype=float32)


In [84]:
tgt=pad_zero(np.array([[5, ] for _ in range(len(a))]), 13)
print(np.array([[5, ] for _ in range(len(a))]))

[[5]
 [5]
 [5]
 [5]
 [5]]


In [10]:
def pad_zero(seqs,max_len):
    matrix=np.zeros((len(seqs),max_len))
    for i,seq in enumerate(seqs):
        matrix[i,:len(seq)]=seq
    return matrix
a=[[1,2,3],[4,5,6]]
print(pad_zero(a,6))

[[1. 2. 3. 0. 0. 0.]
 [4. 5. 6. 0. 0. 0.]]


In [2]:
import numpy as np
import datetime

In [10]:
date_cn=[]
date_en=[]
for i in np.random.randint(40*365*24*3600,60*365*24*3600,5000):
    date=datetime.datetime.fromtimestamp(i)
    date_cn.append(date.strftime("%y-%m-%d"))
    date_en.append(date.strftime("%d/%b/%Y"))




In [16]:
v2i={}
i2v={}
n_vocab=set([str(i) for i in range(10)]+['-','/','<GO>','<EOS>']+
            [item.split('/')[1] for item in date_en])
print(n_vocab)

{'8', 'Jul', '4', 'May', '6', '3', '7', '/', '<EOS>', 'Jan', 'Nov', 'Feb', 'Sep', '1', 'Apr', 'Mar', '2', 'Jun', '<GO>', 'Aug', 'Oct', 'Dec', '0', '9', '5', '-'}


In [19]:
v2i={v:i for i,v in enumerate(sorted(list(n_vocab)),start=1)}
v2i['PAD']=0
i2v={i:v for v,i in v2i.items()}
print(i2v)

{1: '-', 2: '/', 3: '0', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5', 9: '6', 10: '7', 11: '8', 12: '9', 13: '<EOS>', 14: '<GO>', 15: 'Apr', 16: 'Aug', 17: 'Dec', 18: 'Feb', 19: 'Jan', 20: 'Jul', 21: 'Jun', 22: 'Mar', 23: 'May', 24: 'Nov', 25: 'Oct', 26: 'Sep', 0: 'PAD'}


In [21]:
_x=[]
_y=[]
for x,y in zip(date_cn,date_en):
    _x.append([v2i[i] for i in x])
print(_x)
def sample(num):
    idx=np.random.randint(0,len(date_en),num)
    _x=date_cn

[[5, 5, 1, 3, 8, 1, 4, 10], [5, 8, 1, 3, 11, 1, 5, 6], [4, 4, 1, 3, 10, 1, 3, 12], [5, 10, 1, 3, 6, 1, 5, 11], [5, 8, 1, 3, 5, 1, 3, 8], [5, 9, 1, 4, 4, 1, 3, 4], [5, 4, 1, 4, 5, 1, 4, 11], [4, 5, 1, 3, 4, 1, 6, 4], [4, 7, 1, 3, 5, 1, 3, 8], [5, 11, 1, 3, 6, 1, 4, 5], [4, 10, 1, 3, 8, 1, 5, 9], [4, 12, 1, 3, 9, 1, 5, 3], [5, 5, 1, 4, 5, 1, 4, 8], [5, 3, 1, 3, 6, 1, 5, 8], [4, 6, 1, 3, 6, 1, 5, 5], [5, 11, 1, 3, 4, 1, 5, 10], [4, 6, 1, 3, 10, 1, 4, 7], [4, 8, 1, 3, 8, 1, 5, 3], [4, 4, 1, 4, 5, 1, 5, 7], [5, 7, 1, 3, 12, 1, 4, 6], [5, 3, 1, 3, 9, 1, 3, 9], [4, 4, 1, 3, 9, 1, 5, 7], [4, 8, 1, 3, 8, 1, 3, 10], [5, 10, 1, 4, 3, 1, 4, 10], [4, 8, 1, 3, 9, 1, 5, 9], [5, 4, 1, 4, 4, 1, 4, 8], [5, 8, 1, 3, 5, 1, 5, 3], [4, 7, 1, 3, 5, 1, 4, 6], [5, 8, 1, 3, 8, 1, 4, 11], [4, 5, 1, 3, 8, 1, 5, 3], [5, 3, 1, 4, 3, 1, 3, 12], [5, 6, 1, 3, 9, 1, 4, 12], [4, 11, 1, 3, 4, 1, 3, 8], [5, 3, 1, 3, 4, 1, 5, 8], [4, 7, 1, 3, 4, 1, 3, 5], [4, 12, 1, 4, 3, 1, 5, 10], [4, 10, 1, 3, 5, 1, 4, 8], [4, 6, 1, 4, 

In [8]:
a=np.arange(9).reshape((3,3))
a[:,1]=[2,2,2]
print(a.argmax(axis=1))
class code：
    pass
# 获取代码的token列表
def tokenize(code):
    index = 0; # 用来记数 现在处理到第几个字符了
    length = code.length; #代码总字符长度
    print('length', length);

TOKEN_ARRAY = []; #用来存储token的数组
STATUS = '';
while (index < length):
    const ITEM = code[index]; # 应该用switch
    if (getCodeTokenKindFn(ITEM) === TOKEN_KIND.Punctuator):
    #  处理多个符号连成串的情况 例如：==
if(){}
// console.log(code[index], index);
TOKEN_ARRAY.push({
type: TOKEN_KIND.Punctuator,
      value: ITEM
});
index++;
} else if (getCodeTokenKindFn(ITEM) === TOKEN_KIND.WhiteSpace) {
// 空格情况
// console.log(code[index], index);
// TOKEN_ARRAY.push({
//   type: TOKEN_KIND.WhiteSpace,
//   value: ITEM
// });
index++;
} else {
// console.log(code[index], index);
let _S = ITEM;

[1 2 2]
